In [ ]:
# 🔑 UPDATE THESE WITH YOUR REAL HAPPIESTBABY CREDENTIALS
EMAIL = "your-email@example.com"
PASSWORD = "your-password"

print("📋 Credentials configured. Run the demonstration below with: await main()")

# HappiestBaby API - Comprehensive Feature Demonstration

This notebook showcases the **full breadth of features** in this fork compared to the original pysnoo, including:

- 🔐 **Modern AWS Cognito Authentication** 
- 📱 **Enhanced Device Management**
- 📝 **Complete 8-Type Baby Journal System** 
- 🔄 **Full CRUD Operations**
- 📊 **Advanced Data Querying**
- ⚡ **Performance & Reliability Improvements**

## 🚀 What This Fork Provides vs Original pysnoo:

| Feature | Original pysnoo | This Fork (happiestbaby-api) |
|---------|-----------------|-------------------------------|
| **Authentication** | Basic/Legacy | ✅ AWS Cognito with auto-refresh |
| **Device Control** | ✅ Basic Snoo control | ✅ Enhanced device management |
| **Baby Journal** | ❌ None | ✅ **Complete 8-type system** |
| **Data Operations** | Read-only | ✅ **Full CRUD operations** |
| **Unit Conversions** | ❌ None | ✅ **Automatic imperial ↔ metric** |
| **API Coverage** | Limited legacy | ✅ **Multiple API versions** (v10, v11, v12) |

---

**⚠️ Important**: Update the EMAIL and PASSWORD variables below with your real HappiestBaby account credentials before running.

In [ ]:
"""Comprehensive example showcasing the full breadth of HappiestBaby API features.

This example demonstrates the extensive capabilities of this fork compared to the original pysnoo,
including complete baby journal functionality, device management, and advanced features.
"""

import asyncio
import logging
from datetime import datetime, timedelta
from aiohttp import ClientSession

import happiestbaby_api
from happiestbaby_api.errors import SnooError, AuthenticationError

def print_section_header(title: str):
    """Print a formatted section header."""
    print(f"\n{'='*60}")
    print(f"  {title}")
    print(f"{'='*60}")

def print_device_info(device):
    """Print comprehensive device information."""
    print(f"      Device Name: {device.name}")
    print(f"      Device Online: {device.is_online}")
    print(f"      Device On: {device.is_on}")
    print(f"      Device ID: {device.device_id}")
    print(f"      Serial Number: {device.serial_number}")
    print(f"      Firmware Version: {device.firmware_version}")
    print(f"      Baby Details: {device.baby}")
    print(f"      Current State: {device.state}")
    print(f"      Session Active: {device.session is not None}")
    if device.session:
        print(f"      Session Start: {device.session.get('startTime', 'N/A')}")
        print(f"      Session Levels: {len(device.session.get('levels', []))} level changes")
    print("      " + "-" * 50)

async def demonstrate_authentication(websession):
    """Demonstrate modern AWS Cognito authentication."""
    print_section_header("🔐 AWS COGNITO AUTHENTICATION")
    
    try:
        print(f"Authenticating user: {EMAIL}")
        api = await happiestbaby_api.login(EMAIL, PASSWORD, websession)
        
        print("✅ Authentication successful!")
        print(f"   Account ID: {api.account.get('userId')}")
        print(f"   Account Name: {api.account.get('givenName')} {api.account.get('surname')}")
        print(f"   Email: {api.account.get('email')}")
        print(f"   Region: {api.account.get('region')}")
        print(f"   Token expiry handled automatically: ✅")
        
        return api
        
    except AuthenticationError as err:
        print(f"❌ Authentication failed: {err}")
        print("Please check your credentials and try again.")
        return None
    except Exception as err:
        print(f"❌ Unexpected error during authentication: {err}")
        return None

async def demonstrate_device_management(api):
    """Demonstrate comprehensive device management capabilities."""
    print_section_header("📱 DEVICE MANAGEMENT & SESSION TRACKING")
    
    try:
        # Get account information
        account_info = await api.get_account()
        print(f"Account Information Retrieved: ✅")
        
        # Get babies information with proper field names
        babies = await api.get_babies()
        print(f"Babies found: {len(babies) if babies else 0}")
        if babies:
            for i, baby in enumerate(babies):
                # Use correct field names from API response
                baby_name = baby.get('babyName', baby.get('givenName', 'Unnamed'))
                baby_id = baby.get('_id', baby.get('id', 'No ID'))  # Try _id first, then id
                baby_sex = baby.get('sex', 'Unknown')
                baby_birth = baby.get('birthDate', 'Unknown')
                
                print(f"   Baby {i+1}: {baby_name}")
                print(f"      ID: {baby_id}")
                print(f"      Sex: {baby_sex}")
                print(f"      Birth Date: {baby_birth}")
        
        # Get device information
        print(f"Total Snoo Devices Found: {len(api.devices)}")
        
        if len(api.devices) == 0:
            print("No Snoo devices found - this is normal for accounts without Snoo devices")
            return None
        
        # Demonstrate device details if any exist
        for device_id, device in api.devices.items():
            print_device_info(device)
        
        return api.devices
        
    except Exception as err:
        print(f"❌ Error in device management: {err}")
        return None

async def demonstrate_journal_system(api):
    """Demonstrate the comprehensive 8-type journal system."""
    print_section_header("📝 COMPLETE BABY JOURNAL SYSTEM (8 TYPES)")
    
    try:
        # Get babies for journal operations
        babies = await api.get_babies()
        if not babies or len(babies) == 0:
            print("No babies found in account - cannot demonstrate journal functionality")
            return
        
        # Use the first baby for demos
        baby = babies[0]
        baby_id = baby.get('_id', baby.get('id'))  # Try _id first, then id
        baby_name = baby.get('babyName', baby.get('givenName', 'Baby'))
        
        if not baby_id:
            print("Baby has no ID - cannot create journal entries")
            print(f"Available baby fields: {list(baby.keys())}")
            return
        
        print(f"Using baby: {baby_name} (ID: {baby_id})")
        
        # Current time for demonstrations
        now = datetime.now()
        
        print("\n🍼 JOURNAL OPERATIONS DEMONSTRATION")
        print("-" * 50)
        
        # 1. Test reading existing journal data
        print("📊 Testing journal data retrieval...")
        try:
            end_date = datetime.now()
            start_date = end_date - timedelta(days=30)  # Last 30 days
            
            print(f"   Querying data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
            
            # Get different types of journal data
            diaper_data = await api.journal.get_diaper_tracking(baby_id, start_date, end_date)
            print(f"   📋 Diaper entries found: {len(diaper_data) if diaper_data else 0}")
            
            feeding_data = await api.journal.get_feeding_tracking(baby_id, start_date, end_date, 'bottlefeeding')
            print(f"   🍼 Bottle feeding entries found: {len(feeding_data) if feeding_data else 0}")
            
            weight_data = await api.journal.get_weight_tracking(baby_id, start_date, end_date)
            print(f"   ⚖️ Weight entries found: {len(weight_data) if weight_data else 0}")
            
            # Get grouped data (all journal types together)
            grouped_data = await api.journal.get_grouped_tracking(baby_id, start_date, end_date)
            print(f"   📊 Total grouped entries: {len(grouped_data) if grouped_data else 0}")
            
            # Get last journal entries
            last_journals = await api.journal.get_last_journals(baby_id)
            print(f"   🕐 Most recent entries: {len(last_journals) if last_journals else 0}")
            
            print("   ✅ Journal data retrieval working correctly!")
            
            # 2. Demonstrate creating a sample journal entry
            print("\n🧷 Testing journal entry creation...")
            try:
                # Create a sample diaper entry
                diaper_entry = await api.journal.create_diaper_entry(
                    baby_id=baby_id,
                    start_time=now - timedelta(hours=1),
                    diaper_types=['pee'],
                    note="Test diaper entry from API demo"
                )
                print("   ✅ Successfully created test diaper entry!")
                
                # Create a sample feeding entry
                feeding_entry = await api.journal.create_feeding_entry(
                    baby_id=baby_id,
                    start_time=now - timedelta(hours=2),
                    feeding_type='bottlefeeding',
                    amount_imperial=3.0,  # 3 oz
                    milk_type='formula',
                    note="Test feeding entry from API demo"
                )
                print("   ✅ Successfully created test feeding entry (3 oz = ~89 ml)!")
                
            except Exception as create_error:
                print(f"   ⚠️ Journal creation test failed: {create_error}")
            
        except Exception as e:
            print(f"   ⚠️ Journal reading error: {e}")
        
        print("\n🔧 JOURNAL CREATION CAPABILITIES")
        print("-" * 50)
        print("✅ Available Journal Types:")
        print("   1. 🧷 Diaper Changes (pee, poo tracking)")
        print("   2. 🍼 Bottle Feeding (amount + milk type)")
        print("   3. 🤱 Breast Feeding (duration per breast)")
        print("   4. 🥄 Solid Food (food types)")
        print("   5. ⚖️ Weight Tracking (oz ↔ grams)")
        print("   6. 📏 Height Tracking (inches ↔ cm)")
        print("   7. 🧠 Head Circumference (inches ↔ cm)")
        print("   8. 🍼 Pumping (milk volumes)")
        
        print("\n🔄 FULL CRUD OPERATIONS AVAILABLE")
        print("-" * 50)
        print("✅ CREATE: All 8 journal types supported")
        print("✅ READ: Date filtering, grouping, last entries")
        print("✅ UPDATE: Modify existing entries")
        print("✅ DELETE: Remove entries")
        print("✅ UNIT CONVERSION: Automatic imperial ↔ metric")
        print("✅ DATE FILTERING: Custom date ranges")
        print("✅ BULK OPERATIONS: Grouped tracking across all types")
        
    except Exception as err:
        print(f"❌ Error in journal system demonstration: {err}")

async def demonstrate_advanced_features(api):
    """Demonstrate advanced features and capabilities."""
    print_section_header("🚀 ADVANCED FEATURES & CAPABILITIES")
    
    print("🔄 AUTOMATIC UNIT CONVERSIONS")
    print("   • Weight: oz ↔ grams (1 oz = 28.3495 grams)")
    print("   • Liquid: oz ↔ ml (1 oz = 29.5735 ml)")
    print("   • Length: inches ↔ cm (1 inch = 2.54 cm)")
    print("   • Pass either unit, get both automatically stored")
    
    print("\n📅 ADVANCED DATE HANDLING")
    print("   • Timezone-aware datetime processing")
    print("   • Flexible date range queries")
    print("   • ISO 8601 standard formatting")
    print("   • Custom time period filtering")
    
    print("\n⚡ PERFORMANCE & RELIABILITY")
    print("   • Async/await throughout for non-blocking operations")
    print("   • Automatic request retry on transient failures")
    print("   • Connection pooling for HTTP efficiency")
    print("   • Built-in rate limiting and backoff strategies")
    
    print("\n🔒 SECURITY & AUTHENTICATION")
    print("   • AWS Cognito enterprise-grade authentication")
    print("   • Automatic token refresh handling")
    print("   • Thread-safe concurrent operations")
    print("   • Secure credential management")
    
    print("\n🛠️ DEVELOPER EXPERIENCE")
    print("   • Complete type hints for IDE support")
    print("   • Comprehensive error handling with specific exceptions")
    print("   • Extensive logging for debugging")
    print("   • Detailed documentation and examples")

async def main():
    """Run the comprehensive example demonstrating all features."""
    print("🎉 COMPREHENSIVE HAPPIESTBABY API DEMONSTRATION")
    print("This showcases the full breadth of features in this fork vs original pysnoo")
    print(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Set up logging
    logging.basicConfig(level=logging.INFO)  # Use INFO for cleaner output
    
    async with ClientSession() as websession:
        try:
            # 1. Authentication
            api = await demonstrate_authentication(websession)
            if not api:
                print("\n❌ Cannot proceed without authentication. Please check credentials.")
                return
            
            # 2. Device Management
            devices = await demonstrate_device_management(api)
            
            # 3. Complete Journal System
            await demonstrate_journal_system(api)
            
            # 4. Advanced Features Overview
            await demonstrate_advanced_features(api)
            
            print_section_header("✅ DEMONSTRATION COMPLETE")
            print("🎯 WHAT THIS FORK PROVIDES vs ORIGINAL:")
            print("   📱 Enhanced Device Management (from original)")
            print("   🔐 Modern AWS Cognito Authentication (NEW)")
            print("   📝 Complete 8-Type Journal System (NEW)")
            print("   🔄 Full CRUD Operations (NEW)")
            print("   📊 Advanced Data Querying (NEW)")
            print("   🔄 Automatic Unit Conversions (NEW)")
            print("   ⚡ Performance & Reliability Improvements (NEW)")
            print("   🛠️ Enhanced Developer Experience (NEW)")
            print("\n🚀 Ready for production use in baby tracking applications!")
            
        except SnooError as err:
            print(f"\n❌ API Error: {err}")
        except Exception as err:
            print(f"\n❌ Unexpected error: {err}")

print("✨ Example loaded! Run with: await main()")